In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Embedding,Dense, SimpleRNN
from tensorflow.keras import regularizers

In [2]:
import nltk

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
df=pd.read_csv("words.csv")

pd.set_option('display.max_rows', 500)

In [5]:
df.head(5)

words
0               The cat jumped over the fence.
1                The sun was shining brightly.
2  She baked a cake for her friend's birthday.
3           He learned how to play the guitar.
4     The flowers in the garden were blooming.

In [6]:
df.count()

words    81
dtype: int64

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['words'])
vocab_size = len(tokenizer.word_index) + 1
encoded_text = tokenizer.texts_to_sequences(df['words'])
encoded_texts = pad_sequences(encoded_text, padding='post')

In [8]:
word_index = tokenizer.word_index
print(word_index)

{'the': 1, 'in': 2, 'a': 3, 'she': 4, 'and': 5, 'sky': 6, 'was': 7, 'he': 8, 'with': 9, 'through': 10, 'to': 11, 'down': 12, 'on': 13, 'sun': 14, 'wind': 15, 'trees': 16, 'shone': 17, 'her': 18, 'city': 19, 'walked': 20, 'saw': 21, 'of': 22, 'played': 23, 'over': 24, 'learned': 25, 'flowers': 26, 'little': 27, 'rain': 28, 'stars': 29, 'mountains': 30, 'clouds': 31, 'river': 32, 'moon': 33, 'park': 34, 'flowed': 35, 'car': 36, 'girl': 37, 'friends': 38, 'sang': 39, 'landscape': 40, 'brightly': 41, 'for': 42, 'how': 43, 'play': 44, 'guitar': 45, 'boy': 46, 'laughed': 47, 'ran': 48, 'street': 49, 'fell': 50, 'twinkled': 51, 'rose': 52, 'into': 53, 'softly': 54, 'bloomed': 55, 'from': 56, 'night': 57, 'filled': 58, 'people': 59, 'cars': 60, 'market': 61, 'danced': 62, 'music': 63, 'dog': 64, 'swayed': 65, 'read': 66, 'book': 67, 'tree': 68, 'his': 69, 'birds': 70, 'drifted': 71, 'waves': 72, 'shore': 73, 'painted': 74, 'picture': 75, 'shade': 76, 'song': 77, 'cat': 78, 'jumped': 79, 'fence

In [9]:
encoded_texts[:5]

array([[ 1, 78, 79, 24,  1, 80,  0,  0,  0],
       [ 1, 14,  7, 81, 41,  0,  0,  0,  0],
       [ 4, 82,  3, 83, 42, 18, 84, 85,  0],
       [ 8, 25, 43, 11, 44,  1, 45,  0,  0],
       [ 1, 26,  2,  1, 86, 87, 88,  0,  0]])

In [10]:
len(encoded_texts)

81

In [11]:
sequence_length = 4
sequences = []
for i in range(len(encoded_text)):
    seq=[]
    for j in range(len(encoded_text[i]) - sequence_length ):
        seq.append(encoded_text[i][j:j+sequence_length])
    sequences.append(seq)

In [12]:
len(sequences)

81

In [13]:
sequences

[[[1, 78, 79, 24], [78, 79, 24, 1]],
 [[1, 14, 7, 81]],
 [[4, 82, 3, 83], [82, 3, 83, 42], [3, 83, 42, 18], [83, 42, 18, 84]],
 [[8, 25, 43, 11], [25, 43, 11, 44], [43, 11, 44, 1]],
 [[1, 26, 2, 1], [26, 2, 1, 86], [2, 1, 86, 87]],
 [[1, 27, 46, 47],
  [27, 46, 47, 5],
  [46, 47, 5, 48],
  [47, 5, 48, 12],
  [5, 48, 12, 1]],
 [[1, 28, 50, 89], [28, 50, 89, 13], [50, 89, 13, 1]],
 [[1, 15, 7, 91], [15, 7, 91, 2], [7, 91, 2, 1]],
 [[1, 19, 7, 92], [19, 7, 92, 9]],
 [[4, 20, 10, 1],
  [20, 10, 1, 34],
  [10, 1, 34, 5],
  [1, 34, 5, 21],
  [34, 5, 21, 1]],
 [[1, 29, 51, 2], [29, 51, 2, 1]],
 [[1, 30, 52, 53], [30, 52, 53, 1]],
 [[1, 32, 35, 95], [32, 35, 95, 11], [35, 95, 11, 1]],
 [[1, 33, 17, 54], [33, 17, 54, 13], [17, 54, 13, 1]],
 [[1, 36, 98, 12], [36, 98, 12, 1]],
 [[1, 14, 100, 101], [14, 100, 101, 1]],
 [[4, 102, 3, 103], [102, 3, 103, 104], [3, 103, 104, 3], [103, 104, 3, 105]],
 [[8, 25, 11, 44], [25, 11, 44, 1]],
 [[1, 26, 55, 2], [26, 55, 2, 1]],
 [[1, 27, 37, 47],
  [27, 37, 

In [14]:
sequences = list(itertools.chain.from_iterable(sequences))

In [15]:
sequences = pd.DataFrame(sequences)

In [16]:
sequences.shape

(232, 4)

In [17]:
len(word_index)

194

sequences

In [18]:
from keras.layers import LSTM
model =Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=32, input_length=sequence_length))
model.add(LSTM(128, input_shape=(sequences, len(word_index))))
model.add(Dense(500, activation="relu",kernel_regularizer=regularizers.l2(0.005))) 
model.add(Dense(200, activation="relu",kernel_regularizer=regularizers.l2(0.005))) 
model.add(Dense(units=len(tokenizer.word_index)+1, activation='softmax',kernel_regularizer=regularizers.l2(0.005)))
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [19]:
len(tokenizer.word_index)+1

195

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 32)             6240      
                                                                 
 lstm (LSTM)                 (None, 128)               82432     
                                                                 
 dense (Dense)               (None, 500)               64500     
                                                                 
 dense_1 (Dense)             (None, 200)               100200    
                                                                 
 dense_2 (Dense)             (None, 195)               39195     
                                                                 
Total params: 292,567
Trainable params: 292,567
Non-trainable params: 0
_________________________________________________________________


sequences

In [21]:
encoded_text

[[1, 78, 79, 24, 1, 80],
 [1, 14, 7, 81, 41],
 [4, 82, 3, 83, 42, 18, 84, 85],
 [8, 25, 43, 11, 44, 1, 45],
 [1, 26, 2, 1, 86, 87, 88],
 [1, 27, 46, 47, 5, 48, 12, 1, 49],
 [1, 28, 50, 89, 13, 1, 90],
 [1, 15, 7, 91, 2, 1, 16],
 [1, 19, 7, 92, 9, 93],
 [4, 20, 10, 1, 34, 5, 21, 1, 94],
 [1, 29, 51, 2, 1, 6],
 [1, 30, 52, 53, 1, 31],
 [1, 32, 35, 95, 11, 1, 96],
 [1, 33, 17, 54, 13, 1, 97],
 [1, 36, 98, 12, 1, 99],
 [1, 14, 100, 101, 1, 30],
 [4, 102, 3, 103, 104, 3, 105, 37],
 [8, 25, 11, 44, 1, 106],
 [1, 26, 55, 2, 1, 107],
 [1, 27, 37, 47, 5, 108, 12, 1, 109],
 [1, 28, 110, 12, 56, 1, 6],
 [1, 15, 111, 2, 1, 57],
 [1, 19, 7, 58, 9, 59, 5, 60],
 [4, 20, 10, 1, 61, 5, 21, 1, 112],
 [1, 29, 17, 2, 1, 6],
 [1, 30, 113, 114, 53, 1, 31],
 [1, 32, 35, 115, 10, 1, 116],
 [1, 33, 17, 117, 2, 1, 57],
 [4, 62, 11, 1, 118, 22, 1, 63],
 [1, 64, 119, 120],
 [1, 16, 65, 2, 1, 15],
 [1, 36, 121, 122, 123],
 [4, 66, 3, 67, 124, 3, 68],
 [8, 23, 125, 9, 69, 38],
 [1, 70, 39, 126, 2, 1, 16],
 [1, 14, 

In [22]:
len(encoded_text)

81

In [23]:
encoded_len = sequence_length
encoded = []
for i in range(len(encoded_text)):
    for j in range(encoded_len,len(encoded_text[i])):
        encoded.append(encoded_text[i][j])

In [24]:
encoded_text=pd.DataFrame(encoded)

In [25]:
sequences[0]

0       1
1      78
2       1
3       4
4      82
       ..
227     1
228    72
229     4
230    74
231     3
Name: 0, Length: 232, dtype: int64

In [26]:
encoded_text[0]

0       1
1      80
2      41
3      42
4      18
       ..
227     1
228    73
229    22
230     3
231    40
Name: 0, Length: 232, dtype: int64

encoded_texts

encoded_text

sequences

In [27]:
sequences[:10]

0   1   2   3
0   1  78  79  24
1  78  79  24   1
2   1  14   7  81
3   4  82   3  83
4  82   3  83  42
5   3  83  42  18
6  83  42  18  84
7   8  25  43  11
8  25  43  11  44
9  43  11  44   1

In [28]:
# Train the model
model.fit(sequences[:394], encoded_text[:394], epochs=500, batch_size=32)

Epoch 1/500
8/8 [==============================] - 6s 23ms/step - loss: 8.2620 - accuracy: 0.1983
Epoch 2/500
8/8 [==============================] - 0s 12ms/step - loss: 7.4385 - accuracy: 0.2371
Epoch 3/500
8/8 [==============================] - 0s 11ms/step - loss: 6.5201 - accuracy: 0.2371
Epoch 4/500
8/8 [==============================] - 0s 11ms/step - loss: 5.7632 - accuracy: 0.2371
Epoch 5/500
8/8 [==============================] - 0s 11ms/step - loss: 5.3793 - accuracy: 0.2371
Epoch 6/500
8/8 [==============================] - 0s 11ms/step - loss: 5.1039 - accuracy: 0.2371
Epoch 7/500
8/8 [==============================] - 0s 11ms/step - loss: 4.8725 - accuracy: 0.2371
Epoch 8/500
8/8 [==============================] - 0s 11ms/step - loss: 4.6948 - accuracy: 0.2371
Epoch 9/500
8/8 [==============================] - 0s 10ms/step - loss: 4.5507 - accuracy: 0.2371
Epoch 10/500
8/8 [==============================] - 0s 11ms/step - loss: 4.4546 - accuracy: 0.2371
Epoch 11/500
8/8 [=

In [29]:
print(word_index)

{'the': 1, 'in': 2, 'a': 3, 'she': 4, 'and': 5, 'sky': 6, 'was': 7, 'he': 8, 'with': 9, 'through': 10, 'to': 11, 'down': 12, 'on': 13, 'sun': 14, 'wind': 15, 'trees': 16, 'shone': 17, 'her': 18, 'city': 19, 'walked': 20, 'saw': 21, 'of': 22, 'played': 23, 'over': 24, 'learned': 25, 'flowers': 26, 'little': 27, 'rain': 28, 'stars': 29, 'mountains': 30, 'clouds': 31, 'river': 32, 'moon': 33, 'park': 34, 'flowed': 35, 'car': 36, 'girl': 37, 'friends': 38, 'sang': 39, 'landscape': 40, 'brightly': 41, 'for': 42, 'how': 43, 'play': 44, 'guitar': 45, 'boy': 46, 'laughed': 47, 'ran': 48, 'street': 49, 'fell': 50, 'twinkled': 51, 'rose': 52, 'into': 53, 'softly': 54, 'bloomed': 55, 'from': 56, 'night': 57, 'filled': 58, 'people': 59, 'cars': 60, 'market': 61, 'danced': 62, 'music': 63, 'dog': 64, 'swayed': 65, 'read': 66, 'book': 67, 'tree': 68, 'his': 69, 'birds': 70, 'drifted': 71, 'waves': 72, 'shore': 73, 'painted': 74, 'picture': 75, 'shade': 76, 'song': 77, 'cat': 78, 'jumped': 79, 'fence

In [30]:
x=[[1,78,79,23]]

In [31]:
predicted_word = model.predict(x)

1/1 [==============================] - 1s 1s/step


In [32]:
classes=list(range(0,194))

In [33]:
class_name1=classes[np.argmax(predicted_word,axis=1)[0]]

predicted_word

In [34]:
key_list = list(word_index.keys())
val_list = list(word_index.values())
position = val_list.index(class_name1)
print(key_list[position])

with


In [35]:
position

8